In [ ]:
import torchvision.transforms
from torch.utils.data import DataLoader
import torch.utils.data.dataloader
from torch import nn
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torchsummary

Defining train function for training a model

In [ ]:
def train(
        mdl,
        train_dataloader,
        valid_dataloader,
        n_epoch,
        opt,
        criterion,
        valid_step,
        save_path):
    min_valid_loss = float('inf')

    best_state = 0

    # save training and validation losses to draw them later
    trn_losses = []
    vld_losses = []

    train_loss = 0.0
    valid_loss = 0.0


    for e in range(0, n_epoch):
        mdl.train()
        train_loss = 0.0
        for _, (inp, out) in enumerate(train_dataloader):
            opt.zero_grad()
            pred = mdl(inp)
            loss = criterion(pred, out)
            loss.backward()
            opt.step()
            train_loss += loss.item()

        trn_losses.append(train_loss)

        # if it is not time to calculate validation, continue
        if e % valid_step != 0 and e != 0:
            vld_losses.append(valid_loss)
            continue

        mdl.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for i, (inp, out) in enumerate(valid_dataloader):
                pred = mdl(inp)
                ls = criterion(pred, out)
                valid_loss += ls.item()

        vld_losses.append(valid_loss)

        # if they are better weights save them (less loss = better weights)
        if valid_loss < min_valid_loss:
            best_state = mdl.state_dict()
            if save_path != '':
                p = save_path.format(
                    en=n_epoch,
                    vl=valid_loss,
                    tl=train_loss)
                torch.save(mdl, p)

        print('epoch: ' + str(e + 1))
        print('train loss: ' + str(train_loss / len(train_dataloader)))
        print('valid loss: ' + str(valid_loss / len(valid_dataloader)))
        print('\n')

    return trn_losses, vld_losses, best_state


In [ ]:
tns = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
ds = torchvision.datasets.MNIST(root='data/dataset', train=True, download=True, transform=tns)

In [ ]:
N_TRAIN = 10000
N_VALID = 1000
n_test = 2000
train_ds, valid_ds, test_ds, _ = \
    torch.utils.data.random_split(
        ds, [N_TRAIN, N_VALID, n_test, len(ds) - (N_TRAIN + N_VALID + n_test)])

train_dl = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_dl = DataLoader(dataset=valid_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(dataset=test_ds, batch_size=512, shuffle=True)



In [ ]:
en = enumerate(train_dl)
k, (inps, outs) = next(en)
fig = plt.figure()

for i in range(16):
    plt.subplot(4, 4, i+1)
    im = inps[i][0]
    plt.imshow(im, cmap='gray')
    plt.xticks([])
    plt.yticks([])

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2), stride=2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2,2), stride=2),
    nn.Flatten(),
    nn.Linear(800, 10),
    nn.Sigmoid(),
    nn.Linear(10, 10),
    nn.LogSoftmax())

s = torchsummary.summary(model)

In [ ]:
trn_ls, vld_ls, bst_state = train(
    mdl=model,
    train_dataloader=train_dl,
    valid_dataloader=valid_dl,
    n_epoch=10,
    opt=torch.optim.Adam(params=model.parameters(), lr=0.01),
    criterion=nn.CrossEntropyLoss(),
    valid_step=1,
    save_path='')

In [ ]:
plt.plot(trn_ls)
plt.plot(vld_ls)
plt.legend(['train loss', 'validation loss'])
plt.title('training procedure')
plt.show()

Drawing some samples of the trained model
title of each sample is its predicted value
The title color indicated true or false predication(green or red)

In [ ]:
model.state_dict(bst_state)
model.eval()
ROWS = 10
COLS = 10
en = enumerate(test_dl)
_, (inps, outs) = next(en)
with torch.inference_mode():
    preds = model(inps)

fig = plt.figure()
#fig.text('Predicted samples')
for i in range(ROWS*COLS):
    plt.subplot(ROWS, COLS, i+1)
    im = inps[i][0]
    plt.imshow(im, cmap='gray')
    p = torch.argmax(preds[i]).item()
    o = outs[i].item()
    color = 'g'
    if p!=o:
        color = 'r'
    plt.title(str(p), color=color)
    plt.subplots_adjust(top=2)
    plt.xticks([])
    plt.yticks([])
fig.show()

Drawing confusion matrix

In [ ]:
sum = 0.0

preds = []
labels = []
for idx, (inps, outs) in enumerate(test_dl):
    p = model(inps)
    p = torch.argmax(p, 1)
    preds.extend(p.tolist())
    labels.extend(outs.tolist())

cm = confusion_matrix(labels, preds)
ConfusionMatrixDisplay(cm).plot(cmap='gray')

calculating accuracy

In [ ]:
for i in range(len(labels)):
    if labels[i] == preds[i]:
        sum+=1

print('accuracy: ' +
    str(sum/len(test_ds)))